#  **DLT Pipeline**

In [0]:
import dlt
from pyspark.sql.functions import *

###  **Streaming Table**

In [0]:
#Expectations
my_rule = {
    "rule1" : "product_id IS NOT NULL",
    "rule2" : "product_name IS NOT NULL"
    
}

In [0]:
@dlt.table()

@dlt.expect_all_or_drop(my_rule)
def DimProducts_stage():
    df = spark.readStream.table("databricks_cata.silver.products_silver")
    return df

### **Streaming view**

In [0]:
@dlt.view()

def DimProducts_view():
    df = spark.readStream.table("Live.DimProducts_stage")
    return df

### **DimProducts**

In [0]:
dlt.create_streaming_table("DimProducts")

In [0]:
dlt.apply_changes(
  target = "LIVE",
  source = "LIVE.DimProducts_view",
  keys = ["product_id"],
  sequence_by = col("product_id"),
  stored_as_scd_type = 2
)

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import DeltaTable


df = spark.read.table("databricks_cata.silver.products_silver")

df.write.mode("overwrite")\
        .option("path","abfss://gold@databrickssk.dfs.core.windows.net/DimProducts")\
        .saveAsTable("databricks_cata.gold.DimProducts")




